In [1]:
import pandas as pd
import numpy as np
import scipy as sp


## actionType对应的actionTimeDiff

+ 统计1-9分别对应的时间差，最近三个时间差作为提取特征。

+ 这里进行分类计算，[2,3,4,5,6,7,8,9]

In [2]:
df_train_data = pd.read_csv('../data/train/orderFuture_train.csv')
df_test_data = pd.read_csv('../data/test/orderFuture_test.csv')

# 首先选择action信息
df_train_action = pd.read_csv('../data/train/action_train.csv')
df_test_action = pd.read_csv('../data/test/action_test.csv')


In [3]:
def diffInfo(x):
    if len(x) == 0:
        return x
    
    # 所有时间的最大值
    cur = 1505087865
    
    x = list(x)
    x.append(cur)
#     print(x)
    for i in range(len(x)-1):
        x[i] = x[i+1] - x[i]
    
    return np.array(x[:-1])


In [8]:
# 时间间隔特征，[avg,std,var,skew,kurt,min,max, last1, last2,last3,last4,last5]
def getTypeKLast3Diff(df_train_action, Type):
    '''参数：
        df_train_action: 应用数据，可以是train和test
        Type: actionType的值
    
    '''
    
    column = ['userid','type'+str(Type)+'DiffTimeAvg','type'+str(Type)+'DiffTimeMin','type'+str(Type)+'DiffTimeMax',
              'type'+str(Type)+'Last1DiffTime','type'+str(Type)+'Last2DiffTime','type'+str(Type)+'Last3DiffTime']

    df_train_diffTime = pd.DataFrame(columns=column)

    userlist = list(set(df_train_action['userid']))

    k = 0
    for user in userlist:
        k+=1
        if k%5000 == 0:
            print(k)

        df_users_t = df_train_action[df_train_action['userid'] == user]

        # 选择type k 对应的数据
        df_users_t = df_users_t[df_users_t['actionType'] == Type]

        df_users_t = df_users_t.sort_values('actionTime')


        df_users_t['actionTimeDiff'] = diffInfo(df_users_t['actionTime'])

        row = [user]

        ## 所有的差值的最值和平均值
        row.append(df_users_t['actionTimeDiff'].mean())
        row.append(df_users_t['actionTimeDiff'].min())
        row.append(df_users_t['actionTimeDiff'].max())

        # 注意要时间进行发转，统计的时候是到这统计的。
        tmp_time = []
        tmp_time.extend(df_users_t['actionTimeDiff'].values[-3:])
        tmp_time.reverse()

        # 用差值的平均数填充
        if len(tmp_time) < 3:
            tmp_time.extend([row[1] for i in range(3-len(tmp_time))])

        row.extend(tmp_time)

        df_train_diffTime = df_train_diffTime.append(pd.DataFrame([row],columns=column),ignore_index=True)

    # 极差(max-min),(max-avg),(avg-min)
    df_train_diffTime['typeDiff'+str(Type)+'TimeMaxSubMin'] = df_train_diffTime['type'+str(Type)+'DiffTimeMax'] - df_train_diffTime['type'+str(Type)+'DiffTimeMin']
    df_train_diffTime['typeDiff'+str(Type)+'TimeMaxSubAvg'] = df_train_diffTime['type'+str(Type)+'DiffTimeMax'] - df_train_diffTime['type'+str(Type)+'DiffTimeAvg']
    df_train_diffTime['typeDiff'+str(Type)+'TimeAvgSubMin'] = df_train_diffTime['type'+str(Type)+'DiffTimeAvg'] - df_train_diffTime['type'+str(Type)+'DiffTimeMin']
        
    return df_train_diffTime



In [9]:
## train
for Type in [2,3,4,5,6,7,8,9]:
    print(Type)
    df_train_diffTime = getTypeKLast3Diff(df_train_action,Type)
    df_train_data = pd.merge(df_train_data,df_train_diffTime, how='left',on='userid')


2
5000
10000
15000
20000
25000
30000
35000
40000
3
5000
10000
15000
20000
25000
30000
35000
40000
4
5000
10000
15000
20000
25000
30000
35000
40000
5
5000
10000
15000
20000
25000
30000
35000
40000
6
5000
10000
15000
20000
25000
30000
35000
40000
7
5000
10000
15000
20000
25000
30000
35000
40000
8
5000
10000
15000
20000
25000
30000
35000
40000
9
5000
10000
15000
20000
25000
30000
35000
40000


In [18]:
df_test_data = pd.read_csv('../data/test/orderFuture_test.csv')

# 首先选择action信息
df_test_action = pd.read_csv('../data/test/action_test.csv')


In [19]:
## test
for Type in [2,3,4,5,6,7,8,9]:
    print(Type)
    df_test_diffTime = getTypeKLast3Diff(df_test_action,Type)
    df_test_data = pd.merge(df_test_data,df_test_diffTime, how='left',on='userid')


2
5000
10000
3
5000
10000
4
5000
10000
5
5000
10000
6
5000
10000
7
5000
10000
8
5000
10000
9
5000
10000


In [20]:
df_train_data.info()
df_test_data.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 40307 entries, 0 to 40306
Data columns (total 73 columns):
userid                    40307 non-null int64
type2DiffTimeAvg          40307 non-null float64
type2DiffTimeMin          40307 non-null int64
type2DiffTimeMax          40307 non-null int64
type2Last1DiffTime        40307 non-null int64
type2Last2DiffTime        40307 non-null float64
type2Last3DiffTime        40307 non-null float64
typeDiff2TimeMaxSubMin    40307 non-null int64
typeDiff2TimeMaxSubAvg    40307 non-null float64
typeDiff2TimeAvgSubMin    40307 non-null float64
type3DiffTimeAvg          40307 non-null float64
type3DiffTimeMin          40307 non-null int64
type3DiffTimeMax          40307 non-null int64
type3Last1DiffTime        40307 non-null int64
type3Last2DiffTime        40307 non-null float64
type3Last3DiffTime        40307 non-null float64
typeDiff3TimeMaxSubMin    40307 non-null int64
typeDiff3TimeMaxSubAvg    40307 non-null float64
typeDiff3TimeAvgSubMin    4

In [21]:
# 最小的orderTime时间值，用来填充。
MINOREDETIME = 1478363873

MAXORDERTIME = 1505136953

DIFFORDERTIME = MAXORDERTIME - MINOREDETIME

df_train_data.fillna(DIFFORDERTIME,inplace=True)
df_test_data.fillna(DIFFORDERTIME,inplace=True)


In [23]:
# df_train_data.drop('orderType',axis=1,inplace=True)
df_train_data.to_csv('../data/dataSet/df_train_7.csv',index=False,index_label=False)
df_test_data.to_csv('../data/dataSet/df_test_7.csv',index=False,index_label=False)
